In [1]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split

print(f"PyMC version: {pm.__version__}")


PyMC version: 4.2.2


## Ridge classifier with sklearn

In [2]:

X = pd.read_csv('data/features_4_2/hr/45.csv')
df_ = pd.read_csv('data/LookAtMe_002.csv', sep='\t')
y = np.array(list([int (d > 2) for d in df_['rating']]))
y.shape

(160,)

In [3]:
# creating train and test
X_train, X_test,\
y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123)

In [4]:
clf = RidgeClassifier()
clf.fit(X_train, y_train)


RidgeClassifier()

In [5]:
clf.score(X_train, y_train)

0.6470588235294118

In [6]:
clf.score(X_test, y_test)

0.5833333333333334